In [ ]:
import math
import random
import os

calcul de la distance euclidienne définie comme:

  $d_{ij}$: Distance (ou coût) entre les nœuds $i$ et $j$, calculée par la formule euclidienne arrondie :
    \begin{equation}
     d_{ij} = \lfloor \sqrt{(x_j - x_i)^2 + (y_j - y_i)^2} + 0.5 \rfloor
     \end{equation}
    
    

In [ ]:
def compute_dist(xi, xj, yi, yj):
    return math.floor(math.sqrt(math.pow(xi - xj, 2) + math.pow(yi - yj, 2)) + 0.5)

**Calcul de la distance entre dépôt et client**

In [ ]:
def compute_distance_depots(depot_x, depot_y, customers_x, customers_y):
    nb_customers = len(customers_x)
    distance_depots = [None] * nb_customers
    for i in range(nb_customers):
        dist = compute_dist(depot_x, customers_x[i], depot_y, customers_y[i])
        distance_depots[i] = dist
    return distance_depots

**Calcul de la distance entre les clients**

In [ ]:
def compute_distance_matrix(customers_x, customers_y):
    nb_customers = len(customers_x)
    distance_matrix = [[None for _ in range(nb_customers)] for _ in range(nb_customers)]
    for i in range(nb_customers):
        distance_matrix[i][i] = 0
        for j in range(i + 1, nb_customers):
            dist = compute_dist(customers_x[i], customers_x[j], customers_y[i], customers_y[j])
            distance_matrix[i][j] = dist
            distance_matrix[j][i] = dist
    return distance_matrix


**Lecture et extraction des données d'entrée**

In [ ]:
def read_elem(filename):
    with open(filename) as f:
        return [str(elem) for elem in f.read().split()]

def read_input_instances(filename):
    file_it = iter(read_elem(filename))

    nb_customers = int(next(file_it))
    capacity = int(next(file_it))

    demands = [None] * nb_customers
    for i in range(nb_customers):
        demands[i] = int(next(file_it))

    customers_x = [None] * nb_customers
    customers_y = [None] * nb_customers
    depot_x = float(next(file_it))
    depot_y = float(next(file_it))
    for i in range(nb_customers):
        customers_x[i] = float(next(file_it))
        customers_y[i] = float(next(file_it))

    distance_matrix = compute_distance_matrix(customers_x, customers_y)
    distance_depots = compute_distance_depots(depot_x, depot_y, customers_x, customers_y)
    return nb_customers, capacity, distance_matrix, distance_depots, demands

***Implémentation du modèle d'optimisation des routes de livraison avec Variable Neighborhood Search (VNS) ***

In [ ]:
def generate_split_initial_solution(nb_customers, demands, capacity):
    routes = []
    remaining_demands = demands[:]
    while any(remaining_demands):
        route = []
        current_load = 0
        for customer in range(nb_customers):
            if remaining_demands[customer] > 0:
                deliverable = min(remaining_demands[customer], capacity - current_load)
                if deliverable > 0:
                    route.append((customer, deliverable))
                    remaining_demands[customer] -= deliverable
                    current_load += deliverable
                    if current_load == capacity:
                        break
        routes.append(route)
    return routes

def calculate_split_solution_cost(routes, distance_matrix, distance_depots):
    total_cost = 0
    for route in routes:
        if not route:
            continue
        total_cost += distance_depots[route[0][0]]
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i][0]][route[i + 1][0]]
        total_cost += distance_depots[route[-1][0]]
    return total_cost

def generate_neighborhood(routes, nb_customers, demands, capacity):
    neighbors = []
    for i, route in enumerate(routes):
        for j, (customer, quantity) in enumerate(route):
            if quantity > 1:
                new_routes = [list(r) for r in routes]
                split_quantity = quantity // 2
                new_routes[i][j] = (customer, quantity - split_quantity)
                for k, other_route in enumerate(new_routes):
                    if k != i and sum(q for _, q in other_route) + split_quantity <= capacity:
                        new_routes[k].append((customer, split_quantity))
                        neighbors.append(new_routes)
                        break
    return neighbors

def local_search(routes, distance_matrix, distance_depots, demands, capacity):
    current_solution = routes
    best_solution = routes
    best_cost = calculate_split_solution_cost(routes, distance_matrix, distance_depots)

    improving = True
    while improving:
        improving = False
        neighborhood = generate_neighborhood(current_solution, len(demands), demands, capacity)
        for neighbor in neighborhood:
            cost = calculate_split_solution_cost(neighbor, distance_matrix, distance_depots)
            if cost < best_cost:
                best_solution = neighbor
                best_cost = cost
                improving = True
        current_solution = best_solution

    return best_solution

def variable_neighborhood_search(nb_customers, demands, capacity, distance_matrix, distance_depots, max_iterations=100):
    current_solution = generate_split_initial_solution(nb_customers, demands, capacity)
    best_solution = current_solution
    best_cost = calculate_split_solution_cost(current_solution, distance_matrix, distance_depots)

    for iteration in range(max_iterations):
        local_optimum = local_search(current_solution, distance_matrix, distance_depots, demands, capacity)
        local_cost = calculate_split_solution_cost(local_optimum, distance_matrix, distance_depots)

        if local_cost < best_cost:
            best_solution = local_optimum
            best_cost = local_cost

        current_solution = generate_split_initial_solution(nb_customers, demands, capacity)

    return best_solution, best_cost






**Création du fichier Solution**

In [ ]:
def write_solution_to_file(filename, routes, cost, demands):
    with open(filename, "w") as f:
        for i, route in enumerate(routes):
            route_str = "0"
            load = 0
            for customer, quantity in route:
                route_str += f" – {customer + 1} ({quantity})"
                load += quantity
            route_str += " – 0"
            f.write(f"Route {i + 1}: {route_str}\n")
        f.write(f"Total cost: {best_cost}\n")
        f.write(f"Number of deliveries: {sum(len(route) for route in routes)}\n")
        f.write("Loads: " + " ".join(str(sum(q for _, q in route)) for route in routes) + "\n")

In [ ]:

if __name__ == "__main__":
    
    instances_dir = "instances"
    solution_dir="/content/drive/MyDrive/Colab_Notebooks/proj_optim_combin"
    for filename in os.listdir(instances_dir):
        if filename.endswith(".txt"):  
            instance_path = os.path.join(instances_dir, filename)

            nb_customers, capacity, distance_matrix, distance_depots, demands = read_input_instances(instance_path)
            best_solution, best_cost = variable_neighborhood_search(nb_customers, demands, capacity, distance_matrix, distance_depots)

            print(f"Processed file: {filename}")
            print("Best Solution:", best_solution)
            print("Best Cost:", best_cost)

            output_filename = f"solution_{filename}"
            output_path = os.path.join(solution_dir, output_filename)
            write_solution_to_file(output_path, best_solution, best_cost, demands)
            print(f"Solution written to {output_path}")


In [ ]:
import os
import openpyxl

def extract_solution_data(file_path):
    """Extracts the relevant data (last lines) from a solution file."""
    with open(file_path, "r") as f:
        lines = f.readlines()
        if len(lines) >= 4:  
            
            total_cost = lines[-3].split(":")[1].strip()  
            num_deliveries = lines[-2].split(":")[1].strip()  
            loads = lines[-1].split(":")[1].strip()  
            return total_cost, num_deliveries, loads
    return None, None, None  

if __name__ == "__main__":
    solutions_dir = "/content/drive/MyDrive/Colab_Notebooks/proj_optim_combin"  
    
    excel_file_path = os.path.join(solutions_dir, "solutions_summary.xlsx")
    
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Solutions Summary"

    
    sheet.append(["Case", "Total Cost", "Number of Deliveries", "Loads"])

    
    for filename in os.listdir(solutions_dir):
        if filename.endswith(".txt"):  
            solution_path = os.path.join(solutions_dir, filename)

            
            total_cost, num_deliveries, loads = extract_solution_data(solution_path)
            if total_cost and num_deliveries and loads:
                
                case_name = filename.replace("solution_", "").replace(".txt", "")

                
                sheet.append([case_name, total_cost, num_deliveries, loads])

   
    workbook.save(excel_file_path)
    print(f"Solutions summary saved to {excel_file_path}")


In [ ]:
import os
import openpyxl


solutions_dir = "/content/drive/MyDrive/Colab_Notebooks/proj_optim_combin"  

excel_file_path = os.path.join(solutions_dir, "solutions_summary.xlsx")

def extract_solution_data(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()
        if len(lines) >= 4:  
            total_cost = lines[-3].split(":")[1].strip()  
            num_deliveries = lines[-2].split(":")[1].strip()  
            loads = lines[-1].split(":")[1].strip()  
            return total_cost, num_deliveries, loads
    return None, None, None


workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.title = "Solutions Summary"


sheet.append(["Order", "Case", "Total Cost", "Number of Deliveries", "Loads"])


order = 1
for filename in sorted(os.listdir(solutions_dir)):
    if filename.endswith(".txt"):  
        solution_path = os.path.join(solutions_dir, filename)
        total_cost, num_deliveries, loads = extract_solution_data(solution_path)
        if total_cost and num_deliveries and loads:
            case_name = filename.replace("solution_", "").replace(".txt", "")  
            
            sheet.append([order, case_name, total_cost, num_deliveries, loads])
            order += 1

workbook.save(excel_file_path)
excel_file_path
